In [ ]:
# Install the required packages in Jupyter Notebook
!pip install pandas numpy pulp

In [1]:
import pandas as pd
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
data_directory = 'datasets/'
stations = pd.read_csv(f'{data_directory}/station_with_demand.csv')
distance_matrix = pd.read_csv(f'{data_directory}/distance_matrix.csv', index_col=0).to_numpy()

In [4]:
stations

,station_id,station_name,capacity,lng,lat,MEAN_WEEKLY_TRIPS,NORMALISED_CAPACITY,DEMAND_SCORE,DEMAND_CATEGORY
0,5772.05,Morton St & Greenwich St,56,-74.008870,40.731150,0,0.441667,0.000000,Lowest
1,6560.15,Broadway & W 38 St,68,-73.987349,40.752973,511,0.541667,234.719333,Low
2,7727.07,Amsterdam Ave & W 119 St,29,-73.959621,40.808625,0,0.216667,0.000000,Lowest
3,6569.08,W 35 St & Dyer Ave,41,-73.997402,40.754692,206,0.316667,140.972667,Low
4,7023.04,W 59 St & 10 Ave,117,-73.988038,40.770513,341,0.950000,17.391000,Lowest
...,...,...,...,...,...,...,...,...,...
657,8416.10,W 186 St & St Nicholas Ave,26,-73.931308,40.852253,8,0.191667,6.474667,Lowest
658,4846.01,South St & Whitehall St,55,-74.012342,40.701221,313,0.433333,177.679667,Low
659,6890.01,W 53 St & 10 Ave,29,-73.990617,40.766697,424,0.216667,332.557333,Medium
660,6483.06,W 36 St & 7 Ave,38,-73.989539,40.752149,173,0.291667,122.714667,Low


In [4]:
print(distance_matrix)

[[    0.         13914.30811197 41865.43844055 ... 18008.20184989
  13007.34927655  9790.09398049]
 [13914.30811197     0.         27951.13032858 ...  5906.00201928
    906.95883542  4124.21413149]
 [41865.43844055 27951.13032858     0.         ... 23857.23659066
  28858.08916399 32075.34446006]
 ...
 [18008.20184989  5906.00201928 23857.23659066 ...     0.
   5599.49299432  8218.1078694 ]
 [13007.34927655   906.95883542 28858.08916399 ...  5599.49299432
      0.          3217.25529607]
 [ 9790.09398049  4124.21413149 32075.34446006 ...  8218.1078694
   3217.25529607     0.        ]]


In [5]:
# Assigning percentages based on demand category
percentage_map = {
    'Lowest': 0.20,
    'Low': 0.40,
    'Medium': 0.60,
    'Highest': 0.80
}

In [6]:
stations['DEMAND'] = stations['DEMAND_CATEGORY'].map(percentage_map) * stations['capacity']
# stations['DEMAND'] = round(stations['DEMAND'],0).astype(int)

In [7]:
stations

,station_id,station_name,capacity,lng,lat,MEAN_WEEKLY_TRIPS,NORMALISED_CAPACITY,DEMAND_SCORE,DEMAND_CATEGORY,DEMAND
0,5772.05,Morton St & Greenwich St,56,-74.008870,40.731150,0,0.441667,0.000000,Lowest,11.2
1,6560.15,Broadway & W 38 St,68,-73.987349,40.752973,511,0.541667,234.719333,Low,27.2
2,7727.07,Amsterdam Ave & W 119 St,29,-73.959621,40.808625,0,0.216667,0.000000,Lowest,5.8
3,6569.08,W 35 St & Dyer Ave,41,-73.997402,40.754692,206,0.316667,140.972667,Low,16.4
4,7023.04,W 59 St & 10 Ave,117,-73.988038,40.770513,341,0.950000,17.391000,Lowest,23.4
...,...,...,...,...,...,...,...,...,...,...
657,8416.10,W 186 St & St Nicholas Ave,26,-73.931308,40.852253,8,0.191667,6.474667,Lowest,5.2
658,4846.01,South St & Whitehall St,55,-74.012342,40.701221,313,0.433333,177.679667,Low,22.0
659,6890.01,W 53 St & 10 Ave,29,-73.990617,40.766697,424,0.216667,332.557333,Medium,17.4
660,6483.06,W 36 St & 7 Ave,38,-73.989539,40.752149,173,0.291667,122.714667,Low,15.2


In [10]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpInteger

# Function to define the optimization model
def bike_relocation_optimization(stations, demand, weather_scenarios, relocation_costs, capacities, probabilities, weather_factors):
    """
    Solve the bike relocation problem.
    
    Parameters:
    stations (list): List of station names.
    demand (dict): Base demand at each station.
    weather_scenarios (list): List of weather scenarios.
    relocation_costs (dict): Dictionary with relocation cost for each pair of stations (i, j).
    capacities (dict): Dictionary with capacities for each station.
    probabilities (dict): Dictionary with probabilities for each weather scenario.
    weather_factors (dict): Dictionary with weather factors for each station under each scenario.
    
    Returns:
    model (pulp.LpProblem): The solved optimization model.
    """
    # Define the model
    model = LpProblem("Bike_Relocation_Optimization", LpMinimize)
    
    # Decision variables
    # xijk: Number of bikes to move from station i to station j under weather scenario k
    x = LpVariable.dicts("x", ((i, j, k) for i in stations for j in stations for k in weather_scenarios), lowBound=0, cat=LpInteger)
    
    # bik: Number of bikes at station i under weather scenario k
    b = LpVariable.dicts("b", ((i, k) for i in stations for k in weather_scenarios), lowBound=0, cat=LpInteger)
    
    # Objective function: Minimize total relocation cost
    model += lpSum(x[i, j, k] * relocation_costs.get((i, j), 0) * probabilities[k] 
                   for i in stations for j in stations for k in weather_scenarios), "Total_Relocation_Cost"
    
    # Demand constraints: Ensure bike >= expected demand for each station i under scenario k
    for i in stations:
        for k in weather_scenarios:
            expected_demand = demand[i] * weather_factors.get((i, k), 1)
            model += b[i, k] >= expected_demand, f"Demand_Constraint_{i}_{k}"
    
    # Capacity constraints: Ensure bike <= capacity for each station i under scenario k
    for i in stations:
        for k in weather_scenarios:
            model += b[i, k] <= capacities[i], f"Capacity_Constraint_{i}_{k}"
    
    # Flow balance constraints: Final number of bikes at station i equals initial + inflow - outflow
    for i in stations:
        for k in weather_scenarios:
            model += b[i, k] == demand[i] + lpSum(x[j, i, k] for j in stations) - lpSum(x[i, j, k] for j in stations), f"Flow_Balance_{i}_{k}"
    
    # Solve the model
    model.solve()
    
    return model

ModuleNotFoundError: No module named 'pulp'

In [14]:
# Example Input Data (Use your real data here)
stations = ['Station1', 'Station2', 'Station3']
demand = {'Station1': 50, 'Station2': 30, 'Station3': 20}  # Base demand at each station
weather_scenarios = ['Cloudy', 'Rainy', 'Sunny']
probabilities = {'Cloudy': 0.3, 'Rainy': 0.2, 'Sunny': 0.5}
weather_factors = {('Station1', 'Cloudy'): 1.2, ('Station1', 'Rainy'): 1.1, ('Station1', 'Sunny'): 1.0,
                   ('Station2', 'Cloudy'): 1.0, ('Station2', 'Rainy'): 0.9, ('Station2', 'Sunny'): 1.0,
                   ('Station3', 'Cloudy'): 1.1, ('Station3', 'Rainy'): 1.2, ('Station3', 'Sunny'): 1.0}
relocation_costs = {('Station1', 'Station2'): 10, ('Station1', 'Station3'): 15, ('Station2', 'Station3'): 8}  # Cost per bike moved
capacities = {'Station1': 100, 'Station2': 80, 'Station3': 50}  # Capacity of each station

# Call the function to solve the optimization problem
model = bike_relocation_optimization(stations, demand, weather_scenarios, relocation_costs, capacities, probabilities, weather_factors)

# Display the results
if model.status == 1:
    print("Optimization Successful")
    for v in model.variables():
        print(f"{v.name} = {v.varValue}")
else:
    print("Optimization failed")

Optimization failed
